In [ ]:
import requests
from io import StringIO
import pandas as pd
import numpy as np
import sqlite3
sqlite3.register_adapter(np.int64, int)
import re
import math
from datetime import date
import matplotlib.pyplot as plt

# 展開所有dataframe columns
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

# 不縮寫成科學記號
pd.set_option('display.float_format',lambda x : '%.2f' % x)

db = sqlite3.connect('./stock.db' , isolation_level=None)

In [ ]:
df_stock_id_name = pd.read_sql_query("select * from stockIdName",db)

# 宣告一個空的df
df = df_stock_id_name[['id', 'name', 'industry', 'capital']]
df.head()

In [ ]:
def financial_statement(year, season, type='綜合損益彙總表',TYPEK='sii'):
    if year >= 1000:
        year -= 1911

    if type == '綜合損益彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb04'
    elif type == '資產負債彙總表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb05'
    elif type == '現金流量表':
        url = 'https://mops.twse.com.tw/mops/web/ajax_t163sb20'
    else:
        print('type does not match')

    r = requests.post(url, {
        'encodeURIComponent':1,
        'step':1,
        'firstin':1,
        'off':1,
        'TYPEK':TYPEK,  # sii上市, otc上櫃
        'year':str(year),
        'season':str(season),
    })

    r.encoding = 'utf8'
    dfs = pd.read_html(r.text, header=None)
    return pd.concat(dfs[1:], axis=0, sort=False)


In [ ]:
# 檢查目前有幾家公司更新最新財報
f = financial_statement(2024, 3, '綜合損益彙總表', 'sii')
# f = financial_statement(2023, 2, '綜合損益彙總表', 'otc')   # 801

len(f)
# 2023 3 981  2023 4 1004
# 2024 1 1014 2024 2 1014 2024 3 1026

# 綜合損益彙總表

In [ ]:
PROFIT_AND_LOSS = '綜合損益彙總表'

def writeToFinancialStatementDB(df, year, quarter, id, name, revenue, grossProfit, operatingIncome, incomeBeforeTax, income, eps):
    date = f"{year}q{quarter}"
    sql = 'insert into financialStatement (date, id, name, revenue, grossProfit, operatingIncome, incomeBeforeTax, income, qincome, eps, qeps) values (?,?,?,?,?,?,?,?,?,?,?)'
    
    if quarter == 1:
        db.execute(sql, (date, id, name, revenue, grossProfit, operatingIncome, incomeBeforeTax, income, income, eps, eps))
    else:
        try:
            dateDf = df[(df['id']==id) & (df['date']==f'{year}q{quarter-1}')]
            prev_eps = dateDf['eps'].iloc[0]
            prev_income = dateDf['income'].iloc[0]
            qeps = eps - prev_eps
            qincome = income - prev_income
            db.execute(sql, (date, id, name, revenue, grossProfit, operatingIncome, incomeBeforeTax, income, qincome, eps, qeps))
        except:
            db.execute(sql, (date, id, name, revenue, grossProfit, operatingIncome, incomeBeforeTax, income, np.nan, eps, np.nan))
        
def downloadProfitLoss(years, quarters, market='sii'):
    for year in years:
        for quarter in quarters:
            print(f"download {year}q{quarter}")
            historyFinancialStatement = pd.read_sql_query(f"select * from financialStatement order by date",db)
            try:
                df = financial_statement(year, quarter, PROFIT_AND_LOSS, market)
                for index, row in df.iterrows():
                    writeToFinancialStatementDB(historyFinancialStatement,year, quarter, row['公司代號'], row['公司名稱'], row['營業收入'], row['營業毛利（毛損）'], row['營業利益（損失）'], row['稅前淨利（淨損）'], row['本期淨利（淨損）'], row['基本每股盈餘（元）'])
            except:
                print('下載或寫入發生錯誤')


In [ ]:
# 下載綜合損益彙總表 
years = [2024]
quarters = [3]
# sii
downloadProfitLoss(years, quarters)
# otc
downloadProfitLoss(years, quarters, 'otc')

In [ ]:
# search in db
pd.read_sql_query(f"select * from financialStatement where id=2330 order by date",db).tail()

In [ ]:
pd.read_sql_query(f"select * from financialStatement where id=5425 order by date",db)

In [ ]:
# delete duplicates
# pd.read_sql_query(f"delete from financialStatement where rowid not in \
#                   (SELECT min(rowid) from financialStatement GROUP BY id, date)" \
#                   , db)

# t = pd.read_sql_query(f"select * from financialStatement where id=2454 order by date",db)
# t1 = t.drop_duplicates()
# print(len(t), "->", len(t1))

In [ ]:
# delete row
# sql = 'delete from financialStatement where date="2022q4"'
# pd.read_sql_query(sql, db)

In [ ]:
db.close()

# Example 2018Q1 2330
營業收入 248078671\
營業成本 123103977\
營業毛利（毛損）124974694 = 營業收入 - 營業成本 = 248078671 - 123103977\
營業利益（損失）96826946 = 營業毛利（毛損）- 營業費用 - 已實現銷貨（損）益 - 其他收益及費損淨額 = 124974694 - 26728394 - 117155 - 1302199\
稅前淨利（淨損）99943621 = 營業利益（損失）+ 營業外收入及支出 = 96826946 + 3116675\
本期淨利（淨損）89787574 = 稅前淨利（淨損） - 所得稅費用（利益） = 99943621 - 10156047\
淨利（淨損）歸屬於母公司業主 89784622 = 本期淨利（淨損）-  淨利（淨損）歸屬於非控制權益 = 89787574 - 2952


# 資產負債彙總表

In [ ]:
BALANCE_SHEET = '資產負債彙總表'
# sii 權益總額, otc 權益總計 
def downloadBalanceSheet(years, quarters, market='sii'):
    for year in years:
        for quarter in quarters:
            date = f"{year}q{quarter}"
            print(f"download {date} {market}")
            try:
                df = financial_statement(year, quarter, BALANCE_SHEET, market)
                for index, row in df.iterrows():
                    sql = 'insert into balanceSheet (date,id,name,asset,currentAsset,nonCurrentAsset,liabilities,currentLiabilities,nonCurrentLiabilities,additionalPaid,retainedEarning,treasury,shareholderEquity,capital,pb) values (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)'
                    db.execute(sql, (date, row['公司代號'], row['公司名稱'], row['資產總計'], row['流動資產'], row['非流動資產'], row['負債總計'], row['流動負債'], row['非流動負債'], row['資本公積'], row['保留盈餘'], row['庫藏股票'], row['權益總計'], row['股本'], row['每股參考淨值']))
            except Exception as e:
                print('下載或寫入發生錯誤', e)

In [ ]:
years = [2024]
quarters = [1,2,3]
downloadBalanceSheet(years, quarters, 'sii') 
downloadBalanceSheet(years, quarters, 'otc') 

In [ ]:
# check db
sql = f'select * from balanceSheet order by date'
a = pd.read_sql_query(sql, db)
a[a['id']==2330].tail()

In [ ]:
# # delete duplicates
# pd.read_sql_query(f"delete from balanceSheet where rowid not in \
#                   (SELECT min(rowid) from balanceSheet GROUP BY id, date)" \
#                   , db)

# t = pd.read_sql_query(f"select * from balanceSheet where id=2454 order by date",db)
# t1 = t.drop_duplicates()
# print(len(t), "->", len(t1))

In [ ]:
# delete row
sql = 'delete from balanceSheet where date="2023q2"'
# pd.read_sql_query(sql, db)

# 現金流量表

In [ ]:
CASHFLOW = '現金流量表'
def downloadCashFlow(years, quarters, market='sii'):
    for year in years:
        for quarter in quarters:
            date = f"{year}q{quarter}"
            print(f"download {date}", year, quarter, CASHFLOW, market)
            df = financial_statement(year, quarter, CASHFLOW, market)
            sql = 'insert into cashflow (date,id,name,operating,investing,financing,exchange,change,beginning,end, cashflow, qcashflow) values (?,?,?,?,?,?,?,?,?,?,?,?)'
            
            if quarter == 1:
                for index, row in df.iterrows():
                    try:
                        cashflow = int(row["營業活動之淨現金流入（流出）"]) + int(row["投資活動之淨現金流入（流出）"])
                        db.execute(sql, (date, row["公司代號"], row["公司名稱"], row["營業活動之淨現金流入（流出）"], row["投資活動之淨現金流入（流出）"], row["籌資活動之淨現金流入（流出）"], row["匯率變動對現金及約當現金之影響"], row["本期現金及約當現金增加（減少）數"], row["期初現金及約當現金餘額"], row["期末現金及約當現金餘額"], cashflow, cashflow))    
                    except:
                        db.execute(sql, (date, row["公司代號"], row["公司名稱"], row["營業活動之淨現金流入（流出）"], row["投資活動之淨現金流入（流出）"], row["籌資活動之淨現金流入（流出）"], row["匯率變動對現金及約當現金之影響"], row["本期現金及約當現金增加（減少）數"], row["期初現金及約當現金餘額"], row["期末現金及約當現金餘額"], np.nan, np.nan))    
            else:
                df_cashflow = pd.read_sql_query(f'select * from cashflow order by date', db)
                
                for index, row in df.iterrows():
                    try:
                        cashflow = int(row["營業活動之淨現金流入（流出）"]) + int(row["投資活動之淨現金流入（流出）"])
                        prevCashflow = df_cashflow[(df_cashflow['id']==row["公司代號"]) & (df_cashflow['date']==f"{year}q{quarter-1}")]['cashflow'].iloc[0]
                        qcashflow = cashflow - prevCashflow if prevCashflow > 0 else cashflow + prevCashflow
                        db.execute(sql, (date, row["公司代號"], row["公司名稱"], row["營業活動之淨現金流入（流出）"], row["投資活動之淨現金流入（流出）"], row["籌資活動之淨現金流入（流出）"], row["匯率變動對現金及約當現金之影響"], row["本期現金及約當現金增加（減少）數"], row["期初現金及約當現金餘額"], row["期末現金及約當現金餘額"], cashflow, qcashflow))
                    except:
                        db.execute(sql, (date, row["公司代號"], row["公司名稱"], row["營業活動之淨現金流入（流出）"], row["投資活動之淨現金流入（流出）"], row["籌資活動之淨現金流入（流出）"], row["匯率變動對現金及約當現金之影響"], row["本期現金及約當現金增加（減少）數"], row["期初現金及約當現金餘額"], row["期末現金及約當現金餘額"], np.nan, np.nan))


In [ ]:
years = [2024]
quarters = [1,2,3]
downloadCashFlow(years, quarters) 
downloadCashFlow(years, quarters, 'otc') 

In [ ]:
sql = f'select * from cashflow order by date'
a = pd.read_sql_query(sql, db)
# a[a['id']==2330]
a[a['id']==5425].tail()

In [ ]:
# # delete duplicates
# pd.read_sql_query(f"delete from cashflow where rowid not in \
#                   (SELECT min(rowid) from cashflow GROUP BY id, date)" \
#                   , db)

# t = pd.read_sql_query(f"select * from cashflow where id=2454 order by date",db)
# t1 = t.drop_duplicates()
# print(len(t), "->", len(t1))

In [ ]:
# delete row
# sql = 'delete from cashflow where date="2022q1"'
# pd.read_sql_query(sql, db)

In [ ]:
financial_statement(2022, 2, CASHFLOW, 'otc')

# 股本

In [ ]:
# Asset資產, Liabilities負債
BALANCE_SHEET = '資產負債彙總表'
df_AL_sii = financial_statement(2024,3, BALANCE_SHEET)
df_AL_otc = financial_statement(2024,3, BALANCE_SHEET, 'otc')

In [ ]:
print('聯發科股本：', df_AL_sii[df_AL_sii['公司代號'] == 2454]['股本'].iloc[0])
print('中美晶股本：', df_AL_otc[df_AL_otc['公司代號'] == 5483]['股本'].iloc[0])

In [ ]:
# 在已存在table新增 股本 欄位
# db.execute("ALTER TABLE stockIdName ADD COLUMN capital float")

In [ ]:
def writeStockCapital(df):
    for index, row in df_stock_id_name.iterrows():
        stock_id = row['id']
        try:
            stock_capital = df[df['公司代號'] == stock_id]['股本'].iloc[0]
            format_stock_capital = round(stock_capital/1000000,2)
            sql = f"UPDATE stockIdName SET capital = {format_stock_capital} WHERE id = {stock_id}"
            db.execute(sql)
        except:
            continue

writeStockCapital(df_AL_sii)
writeStockCapital(df_AL_otc)

In [ ]:
# read stockIdName table in db
c = pd.read_sql_query("select * from stockIdName",db)
c[c['id']==2330]

In [ ]:
db.close()